# Working with Raster Bands

## Preparing Your Workspace

### Option 1: (recommended) Run in Google Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevinlacaille/presentations/blob/main/scipy2024/4_masks_and_filters.ipynb)

### Option 2: Run local Jupyter instance
You can also choose to open this Notebook in your own local Jupyter instance.

**Prerequisites**

- Install: rasterio
- Download data

In [ ]:
!pip install rasterio
!wget https://raw.githubusercontent.com/kevinlacaille/presentations/main/scipy2024/data/presentation/8928dec4ddbffff/DJI_0876.JPG

In [ ]:
import os

image_path = "/content/DJI_0876.JPG" if os.path.exists(
    "/content/DJI_0876.JPG"
) else "data/presentation/8928dec4ddbffff/DJI_0876.JPG"

In [ ]:
import rasterio

# Open the image and read the bands as numpy arrays
with rasterio.open(image_path) as src:
    blue = src.read(1)
    green = src.read(2)
    red = src.read(3)

In [ ]:
import numpy as np
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# Stack the bands to create a color image
rgb = np.dstack((blue, green, red))

In [ ]:
vari = (green.astype(float) - red.astype(float)) / (
    green.astype(float) + red.astype(float) - blue.astype(float))

## Masking
### Thresholding

This code creates binary masks to distinguish between vegetation and non-vegetation areas by thresholding the VARI index. 

- **Vegetation Mask**: Pixels with VARI values greater than or equal to 0.1 are marked as vegetation (`1`), and others remain `NaN`.
- **Non-Vegetation Mask**: Pixels with VARI values less than 0.1 are marked as non-vegetation (`1`), and others remain `NaN`.

These masks are useful for analyzing and visualizing vegetative regions within the image.

In [ ]:
# Min and max values for the VARI index
vari_min = 0.1
vari_max = 0.5

# Generate the vegetation mask
vegetation_mask = np.full(vari.shape, np.nan)
vegetation_mask[(vari >= vari_min)] = 1

# Generate the non-vegetation mask
non_vegetation_mask = np.full(vari.shape, np.nan)
non_vegetation_mask[vari < vari_min] = 1

In the images below, colored areas are those identified as vegetation in "vegetation mask" and everything but vegation in the "non-vegetation mask".

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

plt.sca(ax[0])
plt.imshow(vegetation_mask, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('vegetation mask')

plt.sca(ax[1])
plt.imshow(non_vegetation_mask, cmap='Blues', vmax=1, vmin=0)
plt.axis('off')
plt.title('non-vegetation mask')

plt.show()


## Filtering

### Smoothing

Apply a Gaussian blur to the vegetation mask to smooth out the edges and reduce noise, providing a cleaner mask for further analysis.

The `GaussianBlur` function from OpenCV uses a kernel size of 7x7 and a standard deviation of 0, resulting in a smoothed version of the vegetation mask.

If the pixel pitch is 2.5 cm, a 7x7 kernel in the Gaussian blur operation covers an area of:

$$ 7 \times 2.5 \text{ cm} \times 7 \times 2.5 \text{ cm} = 17.5 \text{ cm} \times 17.5 \text{ cm} $$

Therefore, a 7x7 kernel translates to a real-world area of 17.5 cm x 17.5 cm (~1/2 a foot x 1/2 a foot). This means that the Gaussian blur operation will smooth the vegetation mask over a square area of approximately 17.5 cm on each side.

In [ ]:
import cv2 as cv

blur = cv.GaussianBlur(vegetation_mask, (7, 7), 0)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

plt.sca(ax[0])
plt.imshow(vegetation_mask, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('vegetation mask')

plt.sca(ax[1])
plt.imshow(blur, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('blur')

plt.show()

### Morphological filtering

Morphological filtering is a set of image processing operations that process images based on their shapes. These operations are particularly effective in removing noise, separating overlapping objects, and enhancing the structure of features in binary and grayscale images.

**Opening Filter**:

A type of morphological operation that consists of an `erosion` followed by a `dilation`. The primary purpose of the opening filter is to remove small objects or noise from the foreground of an image while preserving the shape and size of larger objects.

The 30x30 kernel translates to a real-world area of 75 cm by 75 cm. This means that the morphological opening operation will affect a square area of approximately 75 cm (>2ft) on each side in the image.

**Why use both bluring and morphological filtering?**

- Gaussian blur provides initial noise reduction and smoothing
- Morphological opening further refines the image by removing small, unwanted elements


In [ ]:
opening_kernel = np.ones((30, 30), np.uint8)

opening = cv.morphologyEx(blur, cv.MORPH_OPEN, opening_kernel)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

plt.sca(ax[0])
plt.imshow(rgb)
plt.axis('off')
plt.title('RGB')

plt.sca(ax[1])
plt.imshow(vegetation_mask, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('Vegetation Mask')

plt.sca(ax[2])
plt.imshow(opening, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('Smoothed & Opened Vegetation Mask')

plt.show()

**Closing Filter**:

A type of morphological operation that consists of a `dilation` followed by an `erosion`. The primary purpose of the closing filter is to fill small holes or gaps within the foreground objects while preserving their shape and size.

In [ ]:
closing_kernel = np.ones((30, 30), np.uint8)

closing = cv.morphologyEx(blur, cv.MORPH_CLOSE, closing_kernel)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

plt.sca(ax[0])
plt.imshow(rgb)
plt.axis('off')
plt.title('RGB')

plt.sca(ax[1])
plt.imshow(vegetation_mask, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('Vegetation Mask')

plt.sca(ax[2])
plt.imshow(closing, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('Smoothed & Fully Filtered Vegetation Mask')

plt.show()

**What does the image processing pipeline do?**

1. **Gaussian Blurring**: Reduces overall noise and smooths the image, providing a uniform basis for further analysis.
2. **Morphological Opening**: Removes small noise and artifacts from the foreground, ensuring cleaner separation of important structures.
3. **Morphological Closing**: Fills small holes and gaps within the foreground objects, creating a more solid and continuous representation.

The pipeline achieves a refined and accurate representation of the target features, such as vegetation, in the image. This allows for better analysis and more reliable results in applications like vegetation monitoring and other remote sensing tasks.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

plt.sca(ax[0])
plt.imshow(closing, cmap='Greens', vmax=1, vmin=0)
plt.axis('off')
plt.title('Vegetation Mask')

# Invert the closing mask to get the non-vegetation mask
inverse_closing = np.logical_not(closing.astype(np.uint8)) * 255

plt.sca(ax[1])
plt.imshow(inverse_closing, cmap='Blues', vmax=1, vmin=0)
plt.axis('off')
plt.title('Non-Vegetation Mask')

plt.show()